# L4b: Kernel Regression of Financial Time Series Data
In this lab, we will use [kernel regression](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote14.html) to construct a model of the daily growth rate of an unobserved firm in terms of other members of the [S&P500 index](https://en.wikipedia.org/wiki/S%26P_500). In particular, we'll compute the time series of a specified firm's growth rate using data from a defined list of (observed) basis firms. 

_Hmmm. This sounds familiar._ This idea is similar in spirit to a [state observer from control theory](https://en.wikipedia.org/wiki/State_observer), in the sense we are using (observed) measurements of some states (the growth rate of our basis firms) to estimate the values of (theoretically) unobserved states. However, instead of using a [linear state space model](https://en.wikipedia.org/wiki/State-space_representation), we'll use kernel regression.

### Tasks
Before we start, divide into teams and familiarize yourself with the lab. Then, execute the `Run All Cells` command to check if you (or your neighbor) have any code or setup issues. Code issues, then raise your hands - and let's get those fixed!
* __Task 1: Review Theory of Kernel Regression (10 min)__: Let's take a few minutes to review the theory behind kernel regression and talk about what we can do with it. 
* __Task 2: Setup, Data, and Prerequisites (10 min)__: Nextm, we explore the [Open-High-Low-Close (OHLC) dataset](https://en.wikipedia.org/wiki/Open-high-low-close_chart) we'll explore today. We'll load the full dataset, do some cleanup, scale it, and then split the scaled data into training and test datasets.
* __Task 3: Build, train and test a kernel regression machine (30 min)__: In this task, we'll use the growth rate data for the tickers in a `basis_tickers::Array{String,1}` array, and data for an unknown (to be specified) ticker to estimate the expansion weights $\alpha$. We'll then freeze these weights and perform a growth rate prediction.

## Task 1: Review Theory of Kernel Regression
Suppose we have a dataset $\mathcal{D} = \{(\mathbf{x}_{i},y_{i}) \mid i = 1,2,\dots,n\}$, where the vectors $\mathbf{x}_i \in \mathbb{R}^{m}$ are $m$-dimensional feature vectors ($m\ll{n}$) and the target variables are continuous values $y_i \in \mathbb{R}$, e.g., the price of a house, the price of a stock, the temperature, etc. We can model the relationship between the output, e.g., price, and the features as a linear regression problem: $\hat{\mathbf{y}} = \hat{\mathbf{X}}\theta$ where $\hat{\mathbf{X}}\in\mathbb{R}^{n\times{p}}$ is a data matrix with the augmented feature vectors $\hat{\mathbf{x}}^{\top}$ on the rows, and $\theta\in\mathbb{R}^{p}$ where $p = m+1$ is an unknown parameter vector. The (regularized) least squares solution for the parameters $\theta$ is given by:
$$
\begin{equation}
\hat{\mathbf{\theta}}_{\lambda} = \left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}}+\lambda\,\mathbf{I}\right)^{-1}\hat{\mathbf{X}}^{\top}\mathbf{y}
\end{equation}
$$

The basic idea of kernel regression is to rewrite the parameter vector $\hat{\theta}_{\lambda}$ as a weighted sum of the _augmented training feature variables_: $\hat{\theta}_{\lambda} \equiv \sum_{i=1}^{n}\alpha_{i}\hat{\mathbf{x}}_{i}$. Then for some (new, unknown) feature vector $\hat{\mathbf{z}}$,  the predicted output $\hat{y}$ is given by:
$$
\begin{align}
\hat{y} & = \hat{\mathbf{z}}^{\top}\theta = \sum_{i=1}^{n}\alpha_{i}\left<\hat{\mathbf{z}},\mathbf{x}_{i}\right>\quad\mid\text{\,Replace the inner product with a kernel}\\
        & = \hat{\mathbf{z}}^{\top}\theta \simeq \sum_{i=1}^{n}\alpha_{i}\,k(\hat{\mathbf{z}},\mathbf{x}_{i})
\end{align}
$$
where $k(\hat{\mathbf{z}},\mathbf{x}_{i})$ denotes a kernel function (similarity score) between a new (augmented) feature vector $\hat{\mathbf{z}}$ and the (known) training feature vector $\hat{\mathbf{x}}_{i}$. The $\alpha_{i}$ parameters have a nice [analytical solution](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-4/L4a/docs/Notes.pdf) in terms of the _training data_:
$$
\begin{equation}
\alpha = \left(\mathbf{K}+\lambda\mathbf{I}\right)^{-1}\mathbf{y}
\end{equation}
$$
where $\mathbf{K}$ is the kernel matrix with elements $K_{ij} = k(\mathbf{v}_{i},\mathbf{v}_{j})$, the matrix $\mathbf{I}$ denotes the identity matrix, the vector $\mathbf{y}$ denotes the _training_ outputs and $\lambda\geq{0}$ denotes a regularization parameter. 

## Task 2: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [1]:
include("Include.jl");

  Activating project at `~/Documents/CHEME-5820-Labs-Spring-2025/labs/week-4/L4a`
    Updating `~/Documents/CHEME-5820-Labs-Spring-2025/labs/week-4/L4a/Project.toml`
  [336ed68f] + CSV v0.10.15
  [a93c6f00] + DataFrames v1.7.0
  [31c24e10] + Distributions v0.25.117
  [5789e2e9] + FileIO v1.16.6
  [033835bb] + JLD2 v0.5.11
  [ec8451be] + KernelFunctions v0.10.64
  [91a5bcdd] + Plots v1.40.9
  [08abe8d2] + PrettyTables v2.4.0
  [10745b16] + Statistics v1.11.1
  [f3b207a7] + StatsPlots v0.15.7
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Documents/CHEME-5820-Labs-Spring-2025/labs/week-4/L4a/Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [79e6a3ab] + Adapt v4.1.1
  [66dad0bd] + AliasTables v1.1.3
  [7d9fca2a] + Arpack v0.5.4
  [13072b0f] + AxisAlgorithms v1.1.0
  [d1d4a3ce] + BitFlags v0.1.9
  [336ed68f] + CSV v0.10.15
  [d360d2e6] + ChainRulesCore v1.25.1
  [aaaa29a8] + Clustering v0.15.8
  [944b1d66] + CodecZlib v0.7.8
  [35d6a980] + ColorSchemes v3.28.0
  [3da002f7] + Col

LoadError: LoadError: ArgumentError: Package Colors not found in current path, maybe you meant `import/using .Colors`.
- Otherwise, run `import Pkg; Pkg.add("Colors")` to install the Colors package.
in expression starting at /Users/alinachisti/Documents/CHEME-5820-Labs-Spring-2025/labs/week-4/L4a/Include.jl:16

### Data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2014` until `02-07-2025`, along with data for a few exchange-traded funds and volatility products during that time. We load the `orignal_dataset` by calling the `MyMarketDataSet()` function:

In [2]:
original_dataset = MyMarketDataSet() |> x-> x["dataset"];

LoadError: UndefVarError: `MyMarketDataSet` not defined in `Main`
Suggestion: check for spelling errors or missing imports.


__Clean the data__: Not all tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquisition or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days` variable:

In [3]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow

LoadError: UndefVarError: `original_dataset` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [4]:
dataset = let

    dataset = Dict{String,DataFrame}();
    for (ticker,data) ∈ original_dataset
        if (nrow(data) == maximum_number_trading_days)
            dataset[ticker] = data;
        end
    end
    dataset
end;

LoadError: UndefVarError: `DataFrame` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Let's get a list of firms in the cleaned up `dataset` and save it in the `all_tickers` array. We sort the firms alphabetically from `A` to `Z`:

In [5]:
list_of_all_tickers = keys(dataset) |> collect |> sort;

LoadError: UndefVarError: `dataset` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Compute the expected (annualized) excess log growth rate by passing the `dataset` and the entire list of firms we have in the dataset to the [log_growth_matrix(...) method](src/Compute.jl). The log growth rate between time period $j-1$ to $j$, e.g., yesterday to today is defined as:
$$
\begin{equation}
\mu_{j,j-1} = \left(\frac{1}{\Delta{t}}\right)\ln\left(\frac{S_{j}}{S_{j-1}}\right)
\end{equation}
$$
where $\Delta{t}$ denotes the period time step, and $S_{j}$ denote share price in period $j$.
* The log growth rates are stored in the `D::Array{Float64,2}` variable, a $T-1\times{N}$ array of log return values. Each row of the `D` matrix corresponds to a time value, while each column corresponds to a firm

In [6]:
D = let

    # setup some constants -
    Δt = (1/252); # 1-trading day in units of years
    risk_free_rate = 0.0415; # inferred cc risk-free rate

    # compute
    μ = log_growth_matrix(dataset, list_of_all_tickers, Δt = Δt, 
        risk_free_rate = risk_free_rate);

    # return to caller
    μ
end

LoadError: UndefVarError: `log_growth_matrix` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Next, let's [z-score center](https://en.wikipedia.org/wiki/Feature_scaling) the continous feature data. In [z-score feature scaling](https://en.wikipedia.org/wiki/Feature_scaling), we subtract off the mean of each feature and then divide by the standard deviation, i.e., $x^{\prime} = (x - \mu)/\sigma$ where $x$ is the unscaled data, and $x^{\prime}$ is the scaled data. Under this scaling regime, $x^{\prime}\leq{0}$ will be values that are less than or equal to the mean value $\mu$, while $x^{\prime}>0$ indicate values that are greater than the mean.

We save the z-score centered growth data data in the `D̄::Array{Float64,2}` variable:

In [7]:
D̄ = let

    # setup -
    number_of_examples = size(D,1);

    D̄ = copy(D);
    for j ∈ eachindex(list_of_all_tickers)
        μ = mean(D[:,j]); # compute the mean
        σ = std(D[:,j]); # compute std

        # rescale -
        for k ∈ 1:number_of_examples
            D̄[k,j] = (D[k,j] - μ)/σ;
        end
    end
    
    D̄
end

LoadError: UndefVarError: `D` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Next, (randomly) split that full dataset `D` into `training` and `test` subsets. We do this randomly, where the `number_of_training_examples::Int64` variable specifies the number of training points. The `training::Array{Float64,2}` data will be used to estimate the model parameters, and `test::Array{Float64,2}` will be used for model testing.

In [8]:
training, test = let

    number_of_training_examples = 2000; # set the number to set the number of training examples
    number_of_examples = size(D,1); # number of rows in the full dataset
    full_index_set = range(1,stop=number_of_examples,step=1) |> collect |> Set;
    
    # build index sets for training and testing
    training_index_set = Set{Int64}();
    for i ∈ 1:number_of_training_examples
        push!(training_index_set,i); # use in order
    end
    test_index_set = setdiff(full_index_set,training_index_set);

    # build the test and train datasets -
    training = D̄[training_index_set |> collect,:];
    test = D̄[test_index_set |> collect,:];

    # return
    training, test
end;

LoadError: UndefVarError: `D` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Setup colors

In [9]:
my_color_dictionary = Dict{Int64,RGB}();
my_color_dictionary[1] = colorant"#5e4fa2";
my_color_dictionary[2] = colorant"#3288bd";
my_color_dictionary[3] = colorant"#66c2a5";
my_color_dictionary[4] = colorant"#abdda4";
my_color_dictionary[5] = colorant"#e6f598";
my_color_dictionary[6] = colorant"#fee08b";
my_color_dictionary[7] = colorant"#fdae61";
my_color_dictionary[8] = colorant"#f46d43";
my_color_dictionary[9] = colorant"#d53e4f";
my_color_dictionary[10] = colorant"#9e0142";

## Task 3: Build, train, and test a kernel regression machine
In this task, we'll set up, train, and test a kernel regression machine. First, we'll specify a set of `basis` stocks, i.e., our features and the kernel that we'll use to predict the growth rate of an unknown ticker (output). Then, we'll estimate the expansion parameters $\alpha$ from the `training` data. Finally, we'll run a prediction and compare the predicted versus the observed output using the `test` dataset.

Let's start by specifying _up to_ `10` tickers that we'll use as our basis feature vectors. These can be any firm in the dataset. Specify the `basis` ticker symbols in the `basis_tickers::Array{String,1}` array: 

In [10]:
basis_tickers = ["SPY", "JNJ", "PFE", "AAPL", "MSFT", "GS", "WMT", "AMD"]; # we are going to write are unknown ticker, in terms of these tickers

Next, to speed things up in the simulation loop below, we'll look up the indexes of all `ticker` $\in$ `basis_tickers` and store these in the `basis_ticker_lookup::Dict{Int,Int}` dictionary that will act as a index lookup table.
* __Why a dictionary versus an array__? The [dictionary type](https://docs.julialang.org/en/v1/base/collections/#Base.Dict) offers $\mathcal{O}(1)$ (constant time, i.e., fast) lookup but is expensive to build initially compared to [an array](https://docs.julialang.org/en/v1/base/arrays/#Core.Array-Tuple%7BNothing,%20Any%7D) However, [a dictionary type](https://docs.julialang.org/en/v1/base/collections/#Base.Dict) offers a few other advantages over an array such as the `0` index hack (which I like). Thus, I have a strong preference for dictionaries.

In [11]:
basis_ticker_lookup = let

    # initialize -
    basis_ticker_lookup = Dict{Int,Int}()

    for i ∈ eachindex(basis_tickers)
        j = findfirst(s-> s == basis_tickers[i], list_of_all_tickers); 
        basis_ticker_lookup[i] = j;
    end

    basis_ticker_lookup
end

LoadError: UndefVarError: `list_of_all_tickers` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Visualize
`Unhide` the code block below to see how we plotted the daily growth rate distributions for the firms in the `basis_tickers::Array{String,1}` array for the `training` data. 
* __Summary__: The training distributions will be similar across the different tickers in the `basis_tickers::Array{String,1}` because we z-score centered the data that was used to construct the `training` and `test` datasets. These data show the characteristics of a fat-tailed distribution, i.e., skinny in the center wide of the extremes, i.e., they are _not_ normally distributed (dashed-line).

In [12]:
let

    D = training; # what data are we using?
    number_of_training_days = size(training,1); # number of days of data in training
    number_of_basis_tickers = length(basis_tickers);

    p = plot(bg="gray72", background_color_outside="white", framestyle = :box, fg_legend = :transparent, 
        foreground_color_grid=:black) # make an empty plot
    for i ∈ 1:number_of_basis_tickers
        j = basis_ticker_lookup[i];
        ticker = basis_tickers[i];
        density!(D[:,j], label="$(ticker)", c=my_color_dictionary[i], lw=2)
    end
    current();

    # build a normal -
    i = 4;
    d = fit_mle(Normal, D[:,i])
    plot!(d, ls=:dash, c=:navy, label="Normal $(basis_tickers[i])")
    
    xlabel!("Daily Normalized Growth Rate (AU)", fontsize=18)
    ylabel!("Probability density (AU)", fontsize=18)
end

LoadError: UndefVarError: `training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

`Unhide` the code block below to see how we plotted the daily growth rate time series for the firms in the `basis_tickers::Array{String,1}` array for the training data. There should be `number_of_training_days::Int64` values for each element of the `basis_tickers::Array{String,1}` array.
* __Summary__: The growth time series lie on top of one another but extreme events are visible for various basis tickers, i.e., the spikes. The mean growth rate (because we scaled the data) is around zero, but volatility clustering is still visible in the data, i.e., periods of high volatility are more likely to be followed by high-volatility.

In [13]:
let

    D = training; # what data are we using?
    number_of_training_days = size(training,1); # number of days of data in training
    number_of_basis_tickers = length(basis_tickers);

    p = plot(bg="gray72", background_color_outside="white", framestyle = :box, fg_legend = :transparent); # make an empty plot
    for i ∈ 1:number_of_basis_tickers
        j = basis_ticker_lookup[i];
        ticker = basis_tickers[i];
        plot!(D[:,j], label="$(ticker)", c=my_color_dictionary[i])
    end
    current();

    xlabel!("Trading day index (AU)", fontsize=18)
    ylabel!("Daily Normalized Growth Rate (AU)", fontsize=18)
end

LoadError: UndefVarError: `training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Estimate $\alpha$ parameters
Now, let's use the growth rate data for the tickers in the `basis_tickers::Array{String,1}` array, and data for an unknown (to be specified) ticker to estimate the expansion weights $\alpha$. Let's start by specifying some ticker that we want to model in the `ticker_to_model::String` variable:

In [14]:
ticker_to_model = "NVDA"; # you choose a ticker symbol

Next, specify the kernel that we are going to use. In theory, we can choose [any kernel function exported by the `KernelFunctions.jl` package](https://juliagaussianprocesses.github.io/KernelFunctions.jl/stable/). We've picked a few of the standard kernels below; uncomment the one you want to use.
In addition to choosing the kernel function, you can also specify a value for the [regularization parameter $\lambda>0$](https://en.wikipedia.org/wiki/Regularization_(mathematics)).

In [15]:
λ = 0.5; # pick a regularization parameter  (must be > 0)
γ = 0.5; # pick a scale parameter
k = SqExponentialKernel() ∘ ScaleTransform(γ); # RBF kernel w/a scaling parameter for each basis feature
#k = ExponentialKernel();
#k = LinearKernel(); # inner product

LoadError: UndefVarError: `SqExponentialKernel` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

The $\alpha$ parameters are the expansion weights that appear in the kernel expansion expression: $\hat{y} = \sum_{i=1}^{n}\alpha_{i}\,k(\hat{\mathbf{z}},\hat{\mathbf{x}}_{i})$
where $\hat{y}$ denotes the _predicted_ output corresponsing to the (unkown) input feature vector $\hat{\mathbf{z}}$. From our theory discussion, we know that the $\alpha$ coefficients are given by: $\alpha = \left(\mathbf{K}+\lambda\,\mathbf{I}\right)^{-1}\mathbf{y}$
where the kernel matrix $\mathbf{K}$ has elements $K_{ij} = k(\mathbf{v}_{i},\mathbf{v}_{j})$ computed using the _training data_, the matrix $\mathbf{I}$ denotes the identity matrix, the vector $\mathbf{y}$ denotes the _training_ outputs and $\lambda\geq{0}$ denotes a regularization parameter. 

In the code block below, we:
* After specifying what dataset we are using (and getting some dimensions), we first estimate the vector of observed outputs $\mathbf{y}$ for our target firm from _the training data_.
* Next, we compute the augmented training data matrix $\hat{\mathbf{X}}$ which contains $n^{\prime}$ _training_ feature vectors. The rows of this matrix hold augmented feature vector instances, where each feature vector holds the observed growth rate for the $b$-firms in the `basis_tickers::Array{Float64,1}` array, and a bias term `1` as the last element. Thus, the augmented training data matrix will be $\hat{\mathbf{X}}\in\mathbb{R}^{n^{\prime}\times{(b+1)}}$
* Next, we compute the kernel matrix $\mathbf{K}\in\mathbb{R}^{n^{\prime}\times{n}^{\prime}}$ with elements $K_{ij}=k(\mathbf{v}_{i},\mathbf{v}_{j})$, where $\mathbf{v}_{i}$ and $\mathbf{v}_{j}$ are $b+1$ dimensional feature vectors. __Hmmm__: I thought $\mathbf{K} = \hat{\mathbf{X}}\hat{\mathbf{X}}^{\top}$. What does it mean when use the kernel functions instead of the data to compute $\mathbf{K}$?
* Finally, we compute the expansion weight vector $\mathbf{\alpha}$. This vector will be $\alpha\in\mathbb{R}^{n^{\prime}}$, where each element is the weight of the similarity between a new feature vector $\mathbf{z}$, and the training vectors $\left\{\hat{\mathbf{x}}_{i} \mid i=1,2,\dots,n^{\prime}\right\}$ as measured by the kernel function $k(\hat{\mathbf{z}},\hat{\mathbf{x}}_{i})$.

In [16]:
α, XM = let

    # setup
    D = training; # what data set are we using?
    number_of_training_days = size(training,1); # number of days of data in training set
    
    # 1: compute the training y (output we want model)
    j = findfirst(s-> s == ticker_to_model, list_of_all_tickers); # index of the ticker that we want to model
    y = D[:,j]; # ticker that we want to model -

    # compute the training feature matrix X (instance on rows, training features on cols)
    number_of_basis_tickers = length(basis_tickers);
    XM = zeros(number_of_training_days, number_of_basis_tickers + 1);
    for row ∈ 1:number_of_training_days
        for col ∈ 1:number_of_basis_tickers
            j = basis_ticker_lookup[col];
            XM[row,col] = D[row,j];
        end
        XM[row,end] = 1.0; # augmented vector has a 1 at the end
    end
    
    # compute K, etc -
    K = zeros(number_of_training_days,number_of_training_days);
    for i ∈ 1:number_of_training_days
        vᵢ = XM[i,:];
        for j ∈ 1:number_of_training_days
            vⱼ = XM[j,:];
            K[i,j] = k(vᵢ,vⱼ);
        end
    end
    IM = Matrix(1.0I,number_of_training_days,number_of_training_days);
    
    # compute α -
    α = inv(K+λ*IM)*y;
    
    # # return -
    α, XM
end;

LoadError: UndefVarError: `training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [17]:
size(XM)

LoadError: UndefVarError: `XM` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Predict new output(s) $\hat{\mathbf{y}}$
Now that we have the weights $\alpha$, let's compute a prediction of an unknown growth rate trajectory using the `test` data. We'll save the _predicted_ growth rate values in the `ŷ::Array{Float64,1}` vector, and the _actual_ growth rate in the `y::Array{Float64,1}` vector. Each element of these vectors corresponds to a trading period (in this case, a day, because we are using daily data).

In the code block below, we:
* After we specify some dimensions, i.e., the number of days that we will predict, the number of basis tickers, etc we compute the _actual_ growth rate for our choice of the `ticker_to_model::String`. We save this data in the `y::Array{Float64,1}` variable. Note: these observed values are drawn from `test` and are _not_ used in the prediction.
* Next, we compute the predicted growth rate values, which we save in the `ŷ::Array{Float64,1}` array. For each prediction, we compute the augmented growth rate vector for the basis firms, which we store in the $\mathbf{z}$-array. Then we compute the weighted sum of the kernel terms $k(\hat{\mathbf{z}},\hat{\mathbf{x}}_{i})$, which we store in the `tmp::Float64` term. Finally, we store `tmp` in the `ŷ::Array{Float64,1}` array.

In [18]:
ŷ,y = let

    # setup -
    number_of_prediction_days = 791; # how many days to we want to simulate?
    number_of_training_points = size(training,1);
    number_of_basis_tickers = length(basis_tickers);
    
    # compute actual -
    j = findfirst(s-> s == ticker_to_model, list_of_all_tickers); # index of ticker
    y = zeros(number_of_prediction_days);
    for d ∈ 1:number_of_prediction_days
        y[d] = test[d,j]; # this is what is observed for ticker_to_model
    end

    # compute predicted return -
    ŷ = zeros(number_of_prediction_days);
    for d ∈ 1:number_of_prediction_days

        # generate a *new* feature vector for the basis tickers
        z = zeros(number_of_basis_tickers+1);
        for ticker ∈ 1:number_of_basis_tickers
            j = basis_ticker_lookup[ticker];
            z[ticker] = test[d,j]; # look up values from test for each basis ticker
        end
        z[end] = 1.0; # augmented, has a 1 on the end

        # compute the prediction ŷ - 
        tmp = 0.0;
        for t ∈ 1:number_of_training_points
            xₜ = XM[t,:]; # grab the training feature vector
            tmp+=α[t]*k(z,xₜ); # compare the new basis feature vector to the training vector
        end
        ŷ[d] = tmp;
    end
    ŷ,y
end;

LoadError: UndefVarError: `training` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Visualize
`Unhide` the code block below to see how we plotted the observed and the predicted growth rate time series for the `test` dataset for the ticker specified in the `ticker_to_model::String`.
* __Summary__: Depending upon the choice of kernel and hyperparameters such as the regularization constant $\lambda>0$, the predicted time series (red) generally tracks the observed (blue) time-series. However, the individual ticker may exhibit particular dynamics unique to that asset, which may not be reflected in the basis firms. Thus, we _will likely_ miss significant changes that are unique to a particular asset.

In [19]:
let
    p = plot(bg="gray72", background_color_outside="white", framestyle = :box, fg_legend = :transparent); # make an empty plot
    plot!(y[1:200], label="Actual $(ticker_to_model)", lw=2, c=:navy)
    plot!(ŷ[1:200], label="Predicted $(ticker_to_model)", lw=2, c=:red)
    
    title!("Case: Prediction", fontsize=18)
    xlabel!("Trading day index (AU)", fontsize=18)
    ylabel!("Daily Normalized Growth Rate (AU)", fontsize=18)
end

LoadError: UndefVarError: `y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

`Unhide` the code block below to see how we plotted the daily growth rate distribution for the `ticker_to_model::String` firm for the `test` data. 
* __Summary__: These predicted data show the characteristics of a fat-tailed distribution, i.e., skinny in the center wide of the extremes, i.e., they are _not_ normally distributed (dashed-line). However, we capture the distribution with better or worse quality depending upon our choices. One seemingly conserved feature is that the prediction is even more fat-tailed than the observed distribution.

In [20]:
let
    p = plot(bg="gray72", background_color_outside="white", framestyle = :box, fg_legend = :transparent, legend=:topleft); # make an empty plot
    density!(y, label="Actual $(ticker_to_model)", lw=2, c=:navy)
    density!(ŷ, label="Predicted $(ticker_to_model)", lw=2, c=:red)

    # build a normal -
    d = fit_mle(Normal, y)
    plot!(d, ls=:dash, c=:navy, label="Normal (obs) $(ticker_to_model)")

    title!("Case: Prediction", fontsize=18)
    xlabel!("Daily Normalized Growth Rate (AU)", fontsize=18)
    ylabel!("Probability density (AU)", fontsize=18)
end

LoadError: UndefVarError: `y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Disclaimer and Risks

__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team.

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.